# Introduction to functional data cleaning using nilearn

### Movement is the enemy of neuroimagers

In task-based fMRI, if participants move during task-relevant moments
- Get a huge **false task-related signal** that’s actually due to motion!

In resting-state fMRI, movement can induce **false correlations** between brain regions


Solving for this involves *modelling* our fMRI signal to be comprised of **true brain signal** and **confounder signals**. 


Our goal is to remove a majority (hopefully) of the **confounder signals** and acquire something *closer* to the **true signal**. 

This is achieved via **confound regression**, which is essentially fitting a linear model using confounds as regressors then subtracting it out from the signal


*****

In [ ]:
import os
from nilearn import image as img
from nilearn import plotting as plot
import matplotlib.pyplot as plt
%matplotlib inline

## Step 1: Implementing Confound Regression using FMRIPREP outputs
FMRIPREP estimates confounds for the functional image and outputs it into:

**sub-xxxxx_task-xxxx_space-xxxx_..._confounds.tsv**

Let's load one up and see what it looks like

In [ ]:
func_dir = '../data/func/'
os.listdir(func_dir)

The confounds file is organized like an excel spread-sheet with multiple columns, each for a specific confound.

We can view these using pandas 

In [ ]:
import pandas as pd

In [ ]:
func = os.path.join(func_dir,
                    'sub-10557_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')
confound = os.path.join(func_dir,
                        'sub-10557_task-rest_bold_confounds.tsv'
                        )
confound_df = pd.read_csv(confound,delimiter='\t')

In [ ]:
confound_df.head()

Each of these confounds is computed automatically by fmriprep. The choice of which confounds to use in functional imaging analysis is a source of large debate. We recommend that you check out these sources for a start:

1. https://www.sciencedirect.com/science/article/pii/S1053811917302288#f0005
2. https://www.sciencedirect.com/science/article/pii/S1053811917302288

For now we're going to replicate the pre-processing (mostly) from the seminal Yeo1000 17-networks paper:

https://www.ncbi.nlm.nih.gov/pubmed/21653723

### The (mostly, slightly modified) Yeo 2011 Pre-processing schema

#### Confound regressors
1. 6 motion parameters (X, Y, Z, RotX, RotY, RotZ) 
2. Global signal (GlobalSignal)
3. 2 Largest Principal components of non-grey matter (aCompCor01, aCompCor02)   

This is a total of 9 base confound regressor variables. Finally we add temporal derivatives of each of these signals as well (1 temporal derivative for each), the result is 18 confound regressors.

#### Low/High pass filtering
1. Low pass filtering cutoff: 0.08 
2. High pass filtering cutoff: 0.009

Low pass filters out high frequency signals from our data. fMRI signals are slow evolving processes, any high frequency signals are likely due to noise 
High pass filters out any very low frequency signals (below 0.009Hz), which may be due to intrinsic scanner instabilities

#### Drop dummy TRs
During the initial stages of a functional scan there is a strong signal decay artifact, thus the first 4ish or so 
TRs are very high intensity signals that don't reflect the rest of the scan. Therefore we drop these timepoints. 

#### Censoring + Interpolation (leaving out)
Censoring involves removal and interpolation of high-movement frames from the fMRI data. Interpolation is typically done using sophisticated algorithms much like [Power et al. 2014](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3849338/). 

** We won't be using censoring + interpolation since its fairly complicated and would take up too much time **

***

### Setting up Confound variables for regression

#### Computing temporal derivatives for confound variables

In [ ]:
confound_vars = ['X','Y','Z','RotX','RotY','RotZ','GlobalSignal','aCompCor01','aCompCor02']
confound_df = confound_df[confound_vars]
confound_df.head()

In [ ]:
#For each of these columns we want to compute the temporal derivative, we'll use a for-loop for simplicity
for col in confound_df.columns:
    
    #Example X --> X_dt
    new_name = '{}_dt'.format(col) 
    
    #Compute differences for each pair of rows from start to end. 
    new_col = confound_df[col].diff() 
    
    #Make new column in our dataframe
    confound_df[new_name] = new_col
    

In [ ]:
confound_df.head()

As you might have noticed, we have NaN's in our {confound}_dt. This happens because there is no prior value to the first index to take a difference with, but this isn't a problem since we're going to be dropping 4 timepoints from our data and confounders anyway!

#### Dummy TR Drop


In [ ]:
#First we'll load in our data and check the shape
raw_func_img = img.load_img(func)
raw_func_img.shape

Recall that the fourth dimension represents frames/TRs(timepoints). We want to drop the first four timepoints entirely, to do so we use nibabel's slicer feature. We'll also drop the first 4 confound variable timepoints to match the functional scan

In [ ]:
func_img = raw_func_img.slicer[:,:,:,5:]
func_img.shape

In [ ]:
#Drop confound dummy TRs
drop_confound_df = confound_df.loc[5:]
print(drop_confound_df.shape) #number of rows should match that of the functional image
drop_confound_df.head()

### Applying confound regression

Now we'd like to clean our data of our selected confound variables. This is easily done using nilearn's signal.clean function which looks like:

sgl.clean(signals, confounds, low_pass, high_pass)

Where:
- **signals** represents a matrix of time-series (one column per voxel time-series)
- **confounds** a matrix representing our confound variables (one column per confound)
- **low_pass** the low-pass cutoff
- **high_pass** the high-pass cutoff

This means we need to convert our data and confounds into a matrix format that signal.clean can understand

In [ ]:
from nilearn import signal as sgl

#### Step 1: Set up our data signals matrix
Recall our data is a 4D array, with the fourth dimension represented as time and the other 3 dimensions representing the (x,y,z) coordinate of a particular voxel. We want to convert this to a matrix represented as the following:

$$
\left.\left( 
\vphantom{ \begin{array}{c} 1 \\ 1 \\1 \\1 \\1 \end{array} }
\smash{ \underbrace{
                    \begin{array}{cccccc} 
                    x & x & x & \cdots & x & \\
                    x & x & x & \cdots & x &\\
                    x & x & x & \cdots & x &\\
                    x & x & x & \cdots & x &\\
                    x & x & x & \cdots & x & 
                    \end{array}
                   }_{x*y*z \text{ voxels }}
      }
\right)
\right\}\,t\text{ number of frames}
$$

<br><br>
- The **number of columns represents the total number of voxels (x\*y\*z)**, each column being a single voxel
- The **number of rows represents the number of timepoints**

So we need to *reshape* our data to match this! This is done easily with a single line

In [ ]:
#First we pull out our data as a numpy arrays
signal = func_img.get_data()

#Then we get x,y,z dimensions
x,y,z,t = signal.shape[0],signal.shape[1],signal.shape[2], signal.shape[3]

#Then we get total number of voxels, x*y*z
total_voxels = x*y*z

#Then we reshape to the correct size, note that matrix is flipped on its side
#Where the number of rows matches the number of voxels instead of time-series
shaped_signal = signal.reshape([total_voxels,t])
print(shaped_signal.shape)

In [ ]:
#Now we flip it over (switch columns and rows) 
shaped_signal = shaped_signal.transpose() 
print(shaped_signal.shape) 

#### Step 2: Set up our confound matrix
Recall that we need our confounds matrix to be a matrix with: 
- A row for each timepoint
- A column for each confound variable 

In [ ]:
#Convert our table to a matrix
confound_mat = drop_confound_df.as_matrix()
print(confound_mat.shape) #it is easily converted

#### Step 3: Cleaning our data

In [ ]:
#Set up variables for confound regression
low_pass = 0.08 
high_pass = 0.009

In [ ]:
#Apply!
cleaned_signal = sgl.clean(shaped_signal,confounds=confound_mat,low_pass=low_pass,high_pass=high_pass)

In [ ]:
#Now we can reconstruct our signal, just perform operations in reverse
#Step 1: Flip it back
cleaned_signal = cleaned_signal.transpose() 

#Step 2: Reshape it back into a 4D time-series
cleaned_brain = cleaned_signal.reshape([x,y,z,t])

In [ ]:
import nibabel as nib
import numpy as np

In [ ]:
cleaned_img = nib.Nifti1Image(cleaned_brain,np.eye(4))

### Done!
Hopefully you've learned a bit about how FMRIPREP stores confound regressors and how we can work with it to clean our functional data. 
In the final functional connectivity analysis, we will use the same techniques (except conveniently wrapped versions) to clean all our data and compute seed-based functional connectivity maps